In [1]:
import pandas as pd
import geopandas as gpd
import networkx as nx
from pathlib import Path
from sloyka import Geocoder,TextClassifiers, City_services

INFO: Pandarallel will run on -1 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\thebe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


2024-05-31 14:53:58,426 SequenceTagger predicts: Dictionary with 7 tags: O, S-Service, B-Service, E-Service, I-Service, <START>, <STOP>


In [2]:
df = pd.read_csv("C:\\Users\\thebe\\OneDrive - ITMO UNIVERSITY\\НИРМА\\Data\\test_data\\regional_activity\\comments.csv")

In [3]:
df

,date,id,text,views.count,likes.count,reposts.count,type,link,from_id,post_id,parents_stack,group_name
0,2024-05-22 16:47:48,18024,"Дорого и плохо, а порой еще и опасно Сюжет «7...",1104.0,45,7.0,post,https://vk.com/wall-33468323_41130|«собачий,NaN,NaN,NaN,Шумные Шушары
1,2024-05-22 13:26:45,18023,26 МАЯ | Открытый детский турнир по хоккею Ув...,374.0,3,1.0,post,https://vk.com/mamo_shushary|муниципалитет],NaN,NaN,NaN,Шумные Шушары
2,2024-05-17 16:51:13,18011,Прямой эфир с [id1745844|Екатериной Краевой] с...,2302.0,26,8.0,post,NaN,NaN,NaN,NaN,Шумные Шушары
3,2024-05-16 17:46:54,18003,Показательное отношение Милонова к своим избир...,2515.0,30,11.0,post,https://vk.com/wall-516730_675000|«помощь»],NaN,NaN,NaN,Шумные Шушары
4,2024-05-14 15:46:33,17992,Шушары Today,3022.0,23,1.0,post,NaN,NaN,NaN,NaN,Шумные Шушары
5,2024-05-13 01:05:41,17977,"ВНИМАНИЕ!Сегодня, как обычно, без должного инф...",11194.0,26,34.0,post,NaN,NaN,NaN,NaN,Шумные Шушары
6,2024-05-06 14:10:00,17964,В этом году за счёт муниципального бюджета Шуш...,3149.0,16,17.0,post,NaN,NaN,NaN,NaN,Шумные Шушары
7,2024-04-27 10:27:31,17937,Сегодня свой День рождения отмечает неотъемлем...,7119.0,213,26.0,post,NaN,NaN,NaN,NaN,Шумные Шушары
8,2024-05-22 13:55:20,18025,А что за движуха сегодня утром была на Новгоро...,NaN,3,NaN,comment,NaN,1502930.0,18024.0,NaN,Шумные Шушары
9,2024-05-22 15:16:35,18026,"[id1502930|Анна], таки да, к нам точно едет ""р...",NaN,7,NaN,reply,NaN,967593.0,18024.0,18025.0,Шумные Шушары


In [4]:
g = Geocoder()

2024-05-31 14:54:02,546 SequenceTagger predicts: Dictionary with 5 tags: O, S-ADDRESS, B-ADDRESS, E-ADDRESS, I-ADDRESS


In [5]:
gdf = g.run(osm_id = 338635, tags = {"admin_level": ["8"]}, date = "2024-04-22T00:00:00Z", df=df, text_column='text')

Processing category admin_level: 100%|██████████| 1/1 [00:00<00:00,  3.52it/s]


Error fetching timestamp for osmid 1040824844: Request failed: 404 - Not Found - b''
05-31 14:54 | INFO     | run_OSM_dfs started


C:\Users\thebe\AppData\Local\pypoetry\Cache\virtualenvs\sloyka-C3DMZNjP-py3.11\Lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
C:\Users\thebe\AppData\Local\pypoetry\Cache\virtualenvs\sloyka-C3DMZNjP-py3.11\Lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
C:\Users\thebe\PycharmProjects\sloyka\sloyka\src\geocoder.py:310: UserWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use 

05-31 14:54 | INFO     | find_other_geo_obj started


C:\Users\thebe\AppData\Local\pypoetry\Cache\virtualenvs\sloyka-C3DMZNjP-py3.11\Lib\site-packages\pandas\core\algorithms.py:522: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  common = np.find_common_type([values.dtype, comps_array.dtype], [])
C:\Users\thebe\AppData\Local\pypoetry\Cache\virtualenvs\sloyka-C3DMZNjP-py3.11\Lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
C:\Users\thebe\AppData\Local\pypoetry\Cache\virtualenvs\sloyka-C3DMZNjP-py3.11\Lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  

...

05-31 14:55 | INFO     | extract_toponym started



  0%|          | 0/11 [00:00<?, ?it/s]C:\Users\thebe\AppData\Local\pypoetry\Cache\virtualenvs\sloyka-C3DMZNjP-py3.11\Lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
100%|██████████| 11/11 [00:00<00:00, 245.27it/s]

05-31 14:55 | INFO     | get_stem started


05-31 14:55 | INFO     | create_gdf started


C:\Users\thebe\AppData\Local\pypoetry\Cache\virtualenvs\sloyka-C3DMZNjP-py3.11\Lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
C:\Users\thebe\AppData\Local\pypoetry\Cache\virtualenvs\sloyka-C3DMZNjP-py3.11\Lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
100%|██████████| 15/15 [00:03<00:00,  4.09it/s]


In [6]:
gdf

,date,id,text,views.count,likes.count,reposts.count,type,link,from_id,post_id,...,initial_street,Toponims,full_street_name,location_options,addr_to_geocode,only_full_street_name,Location,geometry,other_geo_obj,geo_obj_tag
0,2024-05-22 16:47:48,18024,"Дорого и плохо, а порой еще и опасно Сюжет «7...",1104.0,45,7.0,post,https://vk.com/wall-33468323_41130|«собачий,NaN,NaN,...,Галицкая,улица,Галицкая улица 4 Санкт-Петербург Россия,['Галицкая улица 4 Санкт-Петербург Россия'],Галицкая улица 4 Санкт-Петербург Россия,Галицкая улица,"4 к1, Галицкая улица, Славянка, Шушары, Санкт-...",POINT (30.46229 59.73173),NaN,street
1,2024-05-22 13:55:20,18025,А что за движуха сегодня утром была на Новгоро...,NaN,3,NaN,comment,NaN,1502930.0,18024.0,...,Новгородском,None,"Новгородский переулок Санкт-Петербург Россия,...",['Новгородский переулок Санкт-Петербург Росси...,Новгородский переулок Санкт-Петербург Россия,Новгородский переулок,"Новгородский переулок, Дибуны, Песочный, Санкт...",POINT (30.13448 60.12044),NaN,street
2,2024-05-22 13:55:20,18025,А что за движуха сегодня утром была на Новгоро...,NaN,3,NaN,comment,NaN,1502930.0,18024.0,...,Новгородском,None,"Новгородский переулок Санкт-Петербург Россия,...",['Новгородский переулок Санкт-Петербург Росси...,Новгородский переулок Санкт-Петербург Россия,Новгородский проспект,"Новгородский переулок, Дибуны, Песочный, Санкт...",POINT (30.13448 60.12044),NaN,street
3,2024-05-22 13:55:20,18025,А что за движуха сегодня утром была на Новгоро...,NaN,3,NaN,comment,NaN,1502930.0,18024.0,...,Новгородском,None,"Новгородский переулок Санкт-Петербург Россия,...",['Новгородский переулок Санкт-Петербург Росси...,Новгородский проспект Санкт-Петербург Россия,Новгородский переулок,"Новгородский проспект, Шушары, Санкт-Петербург...",POINT (30.36738 59.80936),NaN,street
4,2024-05-22 13:55:20,18025,А что за движуха сегодня утром была на Новгоро...,NaN,3,NaN,comment,NaN,1502930.0,18024.0,...,Новгородском,None,"Новгородский переулок Санкт-Петербург Россия,...",['Новгородский переулок Санкт-Петербург Росси...,Новгородский проспект Санкт-Петербург Россия,Новгородский проспект,"Новгородский проспект, Шушары, Санкт-Петербург...",POINT (30.36738 59.80936),NaN,street
5,2024-05-12 22:34:48,17980,"нет, перекресток с Валдайской в ремонте",NaN,0,NaN,comment,NaN,144340.0,17977.0,...,Валдайской,None,Валдайская улица Санкт-Петербург Россия,['Валдайская улица Санкт-Петербург Россия'],Валдайская улица Санкт-Петербург Россия,Валдайская улица,"Валдайская улица, Шушары, Санкт-Петербург, Сев...",POINT (30.37892 59.80403),NaN,street
6,2024-05-15 08:11:43,17999,"[id717545|Аня], Да, выглядит пока не очень. Во...",NaN,0,NaN,reply,NaN,1492921.0,17977.0,...,Витебского,None,"Витебский путепровод Санкт-Петербург Россия,В...",['Витебский путепровод Санкт-Петербург Россия...,Витебский путепровод Санкт-Петербург Россия,Витебский путепровод,"Витебский путепровод, Шушары, Санкт-Петербург,...",POINT (30.38587 59.80856),NaN,street
7,2024-05-15 08:11:43,17999,"[id717545|Аня], Да, выглядит пока не очень. Во...",NaN,0,NaN,reply,NaN,1492921.0,17977.0,...,Витебского,None,"Витебский путепровод Санкт-Петербург Россия,В...",['Витебский путепровод Санкт-Петербург Россия...,Витебский путепровод Санкт-Петербург Россия,Витебский мост,"Витебский путепровод, Шушары, Санкт-Петербург,...",POINT (30.38587 59.80856),NaN,street
8,2024-05-15 08:11:43,17999,"[id717545|Аня], Да, выглядит пока не очень. Во...",NaN,0,NaN,reply,NaN,1492921.0,17977.0,...,Витебского,None,"Витебский путепровод Санкт-Петербург Россия,В...",['Витебский путепровод Санкт-Петербург Россия...,Витебский путепровод Санкт-Петербург Россия,Витебский проспект,"Витебский путепровод, Шушары, Санкт-Петербург,...",POINT (30.38587 59.80856),NaN,street
9,2024-05-15 08:11:43,17999,"[id717545|Аня], Да, выглядит пока не очень. Во...",NaN,0,NaN,reply,NaN,1492921.0,17977.0,...,Витебского,None,"Витебский путепровод Санкт-Петербург Россия,В...",['Витебский путепровод Санкт-Петербург Россия...,Витебский мост Санкт-Петербург Р

In [10]:
gdf['text'].iloc[0]

'Дорого и плохо, а порой еще и опасно  Сюжет «78» о некачественном и дорогом благоустройстве в Петербурге. В этот раз журналисты решили уделить особое внимание нашей местной компании ООО ПСФ «Корт» (и аффилированному с ней ООО «Зеленстрой»).  Её учредителем по документам является бывшая жена Виктора Шабнова, экс-зампредседателя Комитета по благоустройству, уволенного из Смольного в 2018 г. по статье «утрата доверия» за нарушение закона о противодействии коррупции.  Несмотря на такую опалу, семейный бизнес Шабновых вовсю процветает - в последние годы они, без какой-либо конкуренции и снижения цены, забирают все самые жирные контракты по благоустройству в южных районах Петербурга.  Парк Авиаторов (почти 1 млрд рублей), сквер Раисы Штрейс, скейт-парк и [https://vk.com/wall-33468323_41130|«собачий аквапарк»] на Железнодорожной в Пушкине за 78 млн (о котором также упомянуто в этом сюжете), I этап благоустройства «большого круга» в Славянке за 95 млн - список можно бесконечно продолжать. Тол

In [4]:
text_classifier = TextClassifiers('Sandrro/text_to_subfunction_v10')

In [5]:
df[['cats', 'probs']] =  df["text"].progress_map(lambda x: text_classifier.run_text_classifier(x)).to_list()

  0%|          | 0/59 [00:00<?, ?it/s]Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
100%|██████████| 59/59 [01:48<00:00,  1.83s/it]


In [6]:
df

,date,id,text,views.count,likes.count,reposts.count,type,link,from_id,post_id,parents_stack,group_name,cats,probs
0,2024-05-22 16:47:48,18024,"Дорого и плохо, а порой еще и опасно Сюжет «7...",1104.0,45,7.0,post,https://vk.com/wall-33468323_41130|«собачий,NaN,NaN,NaN,Шумные Шушары,Не ЦУР,0.849
1,2024-05-22 13:26:45,18023,26 МАЯ | Открытый детский турнир по хоккею Ув...,374.0,3,1.0,post,https://vk.com/mamo_shushary|муниципалитет],NaN,NaN,NaN,Шумные Шушары,unknown_Благоустройство,0.406
2,2024-05-17 16:51:13,18011,Прямой эфир с [id1745844|Екатериной Краевой] с...,2302.0,26,8.0,post,NaN,NaN,NaN,NaN,Шумные Шушары,unknown_Благоустройство,1.0
3,2024-05-16 17:46:54,18003,Показательное отношение Милонова к своим избир...,2515.0,30,11.0,post,https://vk.com/wall-516730_675000|«помощь»],NaN,NaN,NaN,Шумные Шушары,Жалобы на незаконную торговлю,0.991
4,2024-05-14 15:46:33,17992,Шушары Today,3022.0,23,1.0,post,NaN,NaN,NaN,NaN,Шумные Шушары,Жалобы на незаконную торговлю,0.896
5,2024-05-13 01:05:41,17977,"ВНИМАНИЕ!Сегодня, как обычно, без должного инф...",11194.0,26,34.0,post,NaN,NaN,NaN,NaN,Шумные Шушары,unknown_Транспорт,0.956
6,2024-05-06 14:10:00,17964,В этом году за счёт муниципального бюджета Шуш...,3149.0,16,17.0,post,NaN,NaN,NaN,NaN,Шумные Шушары,"Отсутствие информирования об изменении, закрыт...",0.989
7,2024-04-27 10:27:31,17937,Сегодня свой День рождения отмечает неотъемлем...,7119.0,213,26.0,post,NaN,NaN,NaN,NaN,Шумные Шушары,Не ЦУР,0.519
8,2024-05-22 13:55:20,18025,А что за движуха сегодня утром была на Новгоро...,NaN,3,NaN,comment,NaN,1502930.0,18024.0,NaN,Шумные Шушары,Критика транспортной реформы,0.688
9,2024-05-22 15:16:35,18026,"[id1502930|Анна], таки да, к нам точно едет ""р...",NaN,7,NaN,reply,NaN,967593.0,18024.0,18025.0,Шумные Шушары,Не ЦУР,0.955


In [7]:
percentages = df.copy()['cats'].value_counts(normalize=True)[:5] * 100

In [8]:
percentages_df = pd.DataFrame({'Value': percentages.index, 'Percentage': percentages.values})

In [9]:
percentages_df

,Value,Percentage
0,Не ЦУР,20.338983
1,"Отсутствие информирования об изменении, закрыт...",8.474576
2,unknown_Благоустройство,6.779661
3,"Все другие вопросы строительства, не вошедшие ...",6.779661
4,unknown_Строительство,5.084746


In [10]:
df['toponym'] = 'улица'

In [11]:
user_partition = df['from_id'].loc[df['cats'] == 'Не ЦУР'].value_counts(normalize=True) * 100

In [12]:
user_partition = pd.DataFrame({'user_id': user_partition.index, 'Percentage': user_partition.values})

In [13]:
user_partition

,user_id,Percentage
0,1492921.0,20.0
1,967593.0,10.0
2,636.0,10.0
3,650393846.0,10.0
4,871865.0,10.0
5,185082.0,10.0
6,439008.0,10.0
7,632043.0,10.0
8,144340.0,10.0


In [14]:
df = City_services().run(df, 'text')

In [15]:
df

,date,id,text,views.count,likes.count,reposts.count,type,link,from_id,post_id,parents_stack,group_name,cats,probs,toponym,City_services_extraced,City_services
0,2024-05-22 16:47:48,18024,"Дорого и плохо, а порой еще и опасно Сюжет «7...",1104.0,45,7.0,post,https://vk.com/wall-33468323_41130|«собачий,NaN,NaN,NaN,Шумные Шушары,Не ЦУР,0.849,улица,"[Корт, Парк, аквапарк, Корт, Корт, Корт]","[Морг, Парк, Аквапарк, Морг, Морг, Морг]"
1,2024-05-22 13:26:45,18023,26 МАЯ | Открытый детский турнир по хоккею Ув...,374.0,3,1.0,post,https://vk.com/mamo_shushary|муниципалитет],NaN,NaN,NaN,Шумные Шушары,unknown_Благоустройство,0.406,улица,[],[]
2,2024-05-17 16:51:13,18011,Прямой эфир с [id1745844|Екатериной Краевой] с...,2302.0,26,8.0,post,NaN,NaN,NaN,NaN,Шумные Шушары,unknown_Благоустройство,1.0,улица,[кладбища],[Кладбище]
3,2024-05-16 17:46:54,18003,Показательное отношение Милонова к своим избир...,2515.0,30,11.0,post,https://vk.com/wall-516730_675000|«помощь»],NaN,NaN,NaN,Шумные Шушары,Жалобы на незаконную торговлю,0.991,улица,[],[]
4,2024-05-14 15:46:33,17992,Шушары Today,3022.0,23,1.0,post,NaN,NaN,NaN,NaN,Шумные Шушары,Жалобы на незаконную торговлю,0.896,улица,[],[]
5,2024-05-13 01:05:41,17977,"ВНИМАНИЕ!Сегодня, как обычно, без должного инф...",11194.0,26,34.0,post,NaN,NaN,NaN,NaN,Шумные Шушары,unknown_Транспорт,0.956,улица,[кладбища],[Кладбище]
6,2024-05-06 14:10:00,17964,В этом году за счёт муниципального бюджета Шуш...,3149.0,16,17.0,post,NaN,NaN,NaN,NaN,Шумные Шушары,"Отсутствие информирования об изменении, закрыт...",0.989,улица,[],[]
7,2024-04-27 10:27:31,17937,Сегодня свой День рождения отмечает неотъемлем...,7119.0,213,26.0,post,NaN,NaN,NaN,NaN,Шумные Шушары,Не ЦУР,0.519,улица,[],[]
8,2024-05-22 13:55:20,18025,А что за движуха сегодня утром была на Новгоро...,NaN,3,NaN,comment,NaN,1502930.0,18024.0,NaN,Шумные Шушары,Критика транспортной реформы,0.688,улица,[],[]
9,2024-05-22 15:16:35,18026,"[id1502930|Анна], таки да, к нам точно едет ""р...",NaN,7,NaN,reply,NaN,967593.0,18024.0,18025.0,Шумные Шушары,Не ЦУР,0.955,улица,[],[]


In [16]:
city_services = []
for x in df['City_services'].to_list():
    city_services.extend(x if isinstance(x, list) else [x])
    
city_services = set(city_services)

In [17]:
city_services

{'Аквапарк', 'Кладбище', 'Морг', 'Парк', 'Поле', 'Продукты'}

In [18]:
df['City_services'] = df['City_services'].apply(lambda x: ','.join(set(x)) if x else None)

In [19]:
df['City_services']

0     Аквапарк,Морг,Парк
1                   None
2               Кладбище
3                   None
4                   None
5               Кладбище
6                   None
7                   None
8                   None
9                   None
10                  None
11                  None
12                  None
13                  None
14                  None
15                  None
16                  None
17                  None
18              Продукты
19                  None
20                  None
21                  None
22                  None
23                  None
24                  None
25                  None
26              Кладбище
27                  None
28              Кладбище
29                  None
30                  None
31         Поле,Кладбище
32                  None
33              Кладбище
34                  None
35                  None
36                  None
37                  None
38                  None
39                  None


In [91]:
import numpy as np
df['toponym'] = np.random.randint(0, 3, df.shape[0])

In [92]:
df

,date,id,text,views.count,likes.count,reposts.count,type,link,from_id,post_id,parents_stack,group_name,cats,probs,toponym,City_services_extraced,City_services
0,2024-05-22 16:47:48,18024,"Дорого и плохо, а порой еще и опасно Сюжет «7...",1104.0,45,7.0,post,https://vk.com/wall-33468323_41130|«собачий,NaN,NaN,NaN,Шумные Шушары,Не ЦУР,0.849,1,"[Корт, Парк, аквапарк, Корт, Корт, Корт]","Аквапарк,Морг,Парк"
1,2024-05-22 13:26:45,18023,26 МАЯ | Открытый детский турнир по хоккею Ув...,374.0,3,1.0,post,https://vk.com/mamo_shushary|муниципалитет],NaN,NaN,NaN,Шумные Шушары,unknown_Благоустройство,0.406,0,[],None
2,2024-05-17 16:51:13,18011,Прямой эфир с [id1745844|Екатериной Краевой] с...,2302.0,26,8.0,post,NaN,NaN,NaN,NaN,Шумные Шушары,unknown_Благоустройство,1.0,2,[кладбища],Кладбище
3,2024-05-16 17:46:54,18003,Показательное отношение Милонова к своим избир...,2515.0,30,11.0,post,https://vk.com/wall-516730_675000|«помощь»],NaN,NaN,NaN,Шумные Шушары,Жалобы на незаконную торговлю,0.991,2,[],None
4,2024-05-14 15:46:33,17992,Шушары Today,3022.0,23,1.0,post,NaN,NaN,NaN,NaN,Шумные Шушары,Жалобы на незаконную торговлю,0.896,1,[],None
5,2024-05-13 01:05:41,17977,"ВНИМАНИЕ!Сегодня, как обычно, без должного инф...",11194.0,26,34.0,post,NaN,NaN,NaN,NaN,Шумные Шушары,unknown_Транспорт,0.956,2,[кладбища],Кладбище
6,2024-05-06 14:10:00,17964,В этом году за счёт муниципального бюджета Шуш...,3149.0,16,17.0,post,NaN,NaN,NaN,NaN,Шумные Шушары,"Отсутствие информирования об изменении, закрыт...",0.989,1,[],None
7,2024-04-27 10:27:31,17937,Сегодня свой День рождения отмечает неотъемлем...,7119.0,213,26.0,post,NaN,NaN,NaN,NaN,Шумные Шушары,Не ЦУР,0.519,0,[],None
8,2024-05-22 13:55:20,18025,А что за движуха сегодня утром была на Новгоро...,NaN,3,NaN,comment,NaN,1502930.0,18024.0,NaN,Шумные Шушары,Критика транспортной реформы,0.688,1,[],None
9,2024-05-22 15:16:35,18026,"[id1502930|Анна], таки да, к нам точно едет ""р...",NaN,7,NaN,reply,NaN,967593.0,18024.0,18025.0,Шумные Шушары,Не ЦУР,0.955,2,[],None


In [96]:
router = {0: 'Первый', 1: 'Второй', 2: None}

In [97]:
df['toponym'] = df['toponym'].apply(lambda x: router[x])

In [98]:
top_toponyms = df['toponym'].value_counts(normalize=True) * 100

In [99]:
top_toponyms_names = top_toponyms.index

In [100]:
top_toponyms_df = pd.DataFrame([top_toponyms_names, top_toponyms.to_list()]).T

In [101]:
top_toponyms_df

,0,1
0,Второй,61.111111
1,Первый,38.888889


In [36]:
toponyms = df['toponym'].unique()

In [143]:
toponyms_names = ['Второй']

In [144]:
for i in toponyms_names:
    
    posts_ids = df['id'].loc[df['toponym'] == i].to_list()
    comments_ids = df['id'].loc[df['post_id'].isin(posts_ids) & df['toponym'].isin([i, None])].to_list()
    replies_ids = df['id'].loc[df['parents_stack'].isin(comments_ids) & df['toponym'].isin([i, None])].to_list()
    
    all_ids = tuple(sorted(list(set(posts_ids + comments_ids + replies_ids))))
    
    toponym_df = df.loc[df['id'].isin(all_ids)]
    part_users = len(toponym_df['from_id'].unique())/len(df['from_id'].unique())
    part_messages = len(toponym_df['id'].unique())/len(df['id'].unique())
    
    services = df['City_services_extraced'].loc[df['toponym'] == i]

In [147]:
part_messages

0.4406779661016949

In [158]:
res = []
for i in services:
    res += i
    
res = set(res)

In [159]:
res

{'Корт', 'Парк', 'аквапарк', 'горы', 'кладбища'}

In [161]:
count_dict = {}

for i in res:
    count_dict[i] = 0

In [171]:
df['City_services_extraced'].iloc[0]

['Корт', 'Парк', 'аквапарк', 'Корт', 'Корт', 'Корт']

In [180]:
for i in range(len(df)):
    for j in set(df['City_services_extraced'].iloc[i]):
        if j in count_dict:
            count_dict[j] = count_dict[j] + 1

In [184]:
service_df_list = [[key, value] for key, value in count_dict.items()]

In [189]:
services_df = pd.DataFrame(service_df_list, columns=['service', 'count'])

In [191]:
services_df.sort_values(by='count', ascending=False)

,service,count
4,кладбища,5
0,горы,1
1,Парк,1
2,Корт,1
3,аквапарк,1


In [119]:
toponym_df = df.loc[df['id'].isin(all_ids)]
part_users = len(toponym_df['from_id'].unique())/len(df['from_id'].unique())
part_messages = len(toponym_df['id'].unique())/len(df['id'].unique())

In [120]:
part_users

0.47058823529411764

In [121]:
part_messages

0.3220338983050847